In [1]:
import cv2
import os


import xml.etree.ElementTree as ET

origin_dir = "/app"
cnt = 0

In [2]:
def parse_xml(xml_file):
    """
    Parse the XML file to extract image annotations.
    """
    tree = ET.parse(xml_file)
    root = tree.getroot()
    annotations = {}

    for image in root.findall("image"):
        image_name = image.attrib["name"]
        width = int(image.attrib["width"])
        height = int(image.attrib["height"])
        boxes = []

        for box in image.findall("box"):
            label = box.attrib["label"]
            xtl = int(float(box.attrib["xtl"]))
            ytl = int(float(box.attrib["ytl"]))
            xbr = int(float(box.attrib["xbr"]))
            ybr = int(float(box.attrib["ybr"]))
            boxes.append((label, xtl, ytl, xbr, ybr))
        
        annotations[image_name] = {"width": width, "height": height, "boxes": boxes}
    
    return annotations

def overlay_labels(image_path, annotations, output_dir):
    """
    Overlay labels and bounding boxes on the image and save as JPG.
    """
    global cnt
    image_name = os.path.basename(image_path)
    if image_name not in annotations:
        # print(f"No annotations found for {image_name}. Skipping...")
        return

    image = cv2.imread(image_path)
    data = annotations[image_name]
    
    for box in data["boxes"]:
        label, xtl, ytl, xbr, ybr = box
        # Draw bounding box
        cv2.rectangle(image, (xtl, ytl), (xbr, ybr), (0, 255, 0), 2)
        # Add label text
        cv2.putText(image, label, (xtl, ytl - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Save the image as JPG
    output_file = os.path.join(output_dir, image_name.replace(".png", ".jpg"))
    cv2.imwrite(output_file, image)
    cnt += 1
    if (cnt % 1000 ==0 )  :
        print(f"Saved labeled image to {output_file} /  {cnt})")


def process_files(xml_file, image_dir, output_dir):
    """
    Process XML and PNG files, overlay labels, and save as JPG.
    """
    os.makedirs(output_dir, exist_ok=True)
    annotations = parse_xml(xml_file)

    for file in os.listdir(image_dir):
        if file.endswith(".png"):
            image_path = os.path.join(image_dir, file)
            overlay_labels(image_path, annotations, output_dir)

def process_files_test(xml_file, image_dir, output_dir, img_name):
    """
    Process XML and PNG files, overlay labels, and save as JPG.
    """
    os.makedirs(output_dir, exist_ok=True)
    annotations = parse_xml(xml_file)
    for file in os.listdir(image_dir):
        if (file.endswith(".png")) and (img_name in file):
            image_path = os.path.join(image_dir, file)
            overlay_labels(image_path, annotations, output_dir)


In [3]:
image_dir = f"{origin_dir}/yolo_structure/origin_img/"  # Directory containing PNG files
output_dir = f"{origin_dir}/yolo_structure/adjust_label_img/"  # Directory to save labeled JPG images
xml_file_dir = f"{origin_dir}/yolo_structure/adjust_label_xml/"

for xml_file_name in os.listdir(xml_file_dir):
    if 'ipynb' in xml_file_name:
        continue
    # Run the process
    process_files(xml_file_dir +xml_file_name, image_dir, output_dir)

Saved labeled image to /app/yolo_structure/adjust_label_img/Hongcheon_wonchang4brdg_20201021_1630_WED_15m_NH_highway_OW2_sunny_FHD_119.jpg /  1000)
Saved labeled image to /app/yolo_structure/adjust_label_img/Hongcheon_guljitunnel_20201021_0930_WED_15m_NH_highway_TW2_sunny_FHD_158.jpg /  2000)
Saved labeled image to /app/yolo_structure/adjust_label_img/Cheonan_gangjeong_20201020_0930_TUE_15m_RH_highway_TW3_fog_FHD 1890.jpg /  3000)
Saved labeled image to /app/yolo_structure/adjust_label_img/Hongcheon_guljitunnel_20201020_1200_TUE_15m_NH_highway_TW2_sunny_FHD_103.jpg /  4000)
Saved labeled image to /app/yolo_structure/adjust_label_img/Cheonan_ogsan_20201019_2100_MON_15m_NH_highway_TW3_sunny_FHD_078.jpg /  5000)
Saved labeled image to /app/yolo_structure/adjust_label_img/Cheonan_ogsan_20201021_0900_WED_15m_RH_highway_TW3_sunny_FHD_141.jpg /  6000)
Saved labeled image to /app/yolo_structure/adjust_label_img/Hongcheon_seomgangbrdg_20201020_0930_TUE_15m_NH_highway_TW2_sunny_FHD_093.jpg /  70

In [4]:
image_dir = f"{origin_dir}/yolo_structure/origin_img/"  # Directory containing PNG files
output_dir = f"{origin_dir}/yolo_structure/adjust_label_img/"  # Directory to save labeled JPG images
xml_file_dir = f"{origin_dir}/yolo_structure/adjust_label_xml/"

for xml_file_name in os.listdir(xml_file_dir):
    if 'ipynb' in xml_file_name:
        continue
    # Run the process
    process_files_test(xml_file_dir + xml_file_name, image_dir, output_dir, "Gangneung_wichon2brdg_20201021_1300_WED_15m_NH_highway_TW2_sunny_FHD")
    # process_files(xml_file_name, image_dir, output_dir)

Saved labeled image to /app/yolo_structure/adjust_label_img/Gangneung_wichon2brdg_20201021_1300_WED_15m_NH_highway_TW2_sunny_FHD_163.jpg /  19000)


In [ ]:
Cheonan_ogsan_20201020_1230_TUE_15m_NH_highway_TW3_sunny_FHD

In [6]:
xml_file_dir = f"{origin_dir}/yolo_structure/adjust_label_xml/"
xml_area_l = []
for xml_file_name in os.listdir(xml_file_dir):

    adjust_xml_file = f"{xml_file_dir}/{xml_file_name}"

    tree = ET.parse(adjust_xml_file)
    root = tree.getroot()
    objects = []
    for obj in root.findall("object"):
        label = obj.find("name").text
        bbox = obj.find("bndbox")
        xmin = int(bbox.find("xmin").text)
        ymin = int(bbox.find("ymin").text)
        xmax = int(bbox.find("xmax").text)
        ymax = int(bbox.find("ymax").text)
        objects.append((label, xmin, ymin, xmax, ymax))
        print(label)
    


Cheonan_ogsan_20201020_1230_TUE_15m_NH_highway_TW3_sunny_FHD.xml
.ipynb_checkpoints


In [32]:
xml_file = "adjust_label_xml/Suwon_CH01_20200720_1730_MON_9m_NH_highway_TW5_sunny_FHD.xml"

In [33]:
tree = ET.parse(xml_file)
root = tree.getroot()
objects = []
for obj in root.findall("object"):
    label = obj.find("name").text
    bbox = obj.find("bndbox")
    xmin = int(bbox.find("xmin").text)
    ymin = int(bbox.find("ymin").text)
    xmax = int(bbox.find("xmax").text)
    ymax = int(bbox.find("ymax").text)
    objects.append((label, xmin, ymin, xmax, ymax))
    print(label)


In [34]:
def parse_xml(xml_file):
    """
    Parse the XML file to extract image annotations.
    """
    tree = ET.parse(xml_file)
    root = tree.getroot()
    annotations = {}

    for image in root.findall("image"):
        image_name = image.attrib["name"]
        width = int(image.attrib["width"])
        height = int(image.attrib["height"])
        boxes = []

        for box in image.findall("box"):
            label = box.attrib["label"]
            xtl = int(float(box.attrib["xtl"]))
            ytl = int(float(box.attrib["ytl"]))
            xbr = int(float(box.attrib["xbr"]))
            ybr = int(float(box.attrib["ybr"]))
            boxes.append((label, xtl, ytl, xbr, ybr))
        
        annotations[image_name] = {"width": width, "height": height, "boxes": boxes}
    
    return annotations

def overlay_labels(image_path, annotations, output_dir):
    """
    Overlay labels and bounding boxes on the image and save as JPG.
    """
    image_name = os.path.basename(image_path)
    if image_name not in annotations:
        print(f"No annotations found for {image_name}. Skipping...")
        return

    image = cv2.imread(image_path)
    data = annotations[image_name]
    
    for box in data["boxes"]:
        label, xtl, ytl, xbr, ybr = box
        # Draw bounding box
        cv2.rectangle(image, (xtl, ytl), (xbr, ybr), (0, 255, 0), 2)
        # Add label text
        cv2.putText(image, label, (xtl, ytl - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Save the image as JPG
    output_file = os.path.join(output_dir, image_name.replace(".png", ".jpg"))
    cv2.imwrite(output_file, image)
    print(f"Saved labeled image to {output_file}")

In [35]:

def process_files(xml_file, image_dir, output_dir):
    """
    Process XML and PNG files, overlay labels, and save as JPG.
    """
    os.makedirs(output_dir, exist_ok=True)
    annotations = parse_xml(xml_file)

    for file in os.listdir(image_dir):
        if file.endswith(".png"):
            image_path = os.path.join(image_dir, file)
            overlay_labels(image_path, annotations, output_dir)

image_dir = "./origin_img/"  # Directory containing PNG files
output_dir = "./adjust_label_img/"  # Directory to save labeled JPG images

# Run the process
process_files(xml_file, image_dir, output_dir)

Saved labeled image to ./adjust_label_img/Suwon_CH01_20200720_1730_MON_9m_NH_highway_TW5_sunny_FHD_014.jpg
Saved labeled image to ./adjust_label_img/Suwon_CH01_20200720_1730_MON_9m_NH_highway_TW5_sunny_FHD_009.jpg
Saved labeled image to ./adjust_label_img/Suwon_CH01_20200720_1730_MON_9m_NH_highway_TW5_sunny_FHD_006.jpg
Saved labeled image to ./adjust_label_img/Suwon_CH01_20200720_1730_MON_9m_NH_highway_TW5_sunny_FHD_002.jpg
Saved labeled image to ./adjust_label_img/Suwon_CH01_20200720_1730_MON_9m_NH_highway_TW5_sunny_FHD_003.jpg
